In [1]:
import pandas as pd
import numpy as np
import datetime
from scorepi import *
from epiweeks import Week
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
locations = pd.read_csv('./dat/locations.csv',dtype={'location':str})

In [156]:
class ScenarioEnsemble:
    # get the median aggregate scenario ensemble for an individual model given a set of projections for >1 scenario
    # reported in quantile format.
    
    # this method takes the average of all quantiles to find one projection that is an ensemble of all scenarios.
    # performed for all projection locations reported in the given DataFrame
    
    # Input dataframe must have: location, scenario_id, target_end_date, target, value, type, and quantile columns
    
    def __init__(self, df, obsdf, target, incidence = True, max_date = False, start_week = False, end_week = False):
        self.df = df # input dataframe with all scenarios, locations, and quantiles
        self.obsdf = obsdf # input of surveillance data of interest
        self.target = target # target metric of interest (case, death, hospitalization)
        self.inc = incidence # True if incident measures, False if cumulative
        self.max_date = max_date # maximum date you want to analyze, cut off date
        self.start_week = start_week # beginning of observations of interest
        self.end_week = end_week # end of observations of interest
        self.locations = pd.DataFrame()
        self.scenario_ensemble = pd.DataFrame()
        
        
    def get_locations(self):
        # get df with US state names, populations, and abbreviations and corresponding numerical code 
        locations = pd.read_csv('./dat/locations.csv',dtype={'location':str})
        
        self.locations = locations
        
        
    def get_observations(self, target_location):
        # get and format surveillance data of interest
        observations = self.obsdf.copy()
        
        if self.target == 'hosp':
            target_obs = 'hospitalization'
        else:
            target_obs = self.target
            
        # read in observations dataframe
        observations = self.obsdf.copy()
        observations['date'] = pd.to_datetime(observations['date'])

        #filter start - end week
        if self.start_week:
            observations = observations[(observations['date'] >= pd.to_datetime(self.start_week.startdate())) ]
            
        if self.end_week:
            observations = observations[(observations['date'] <= pd.to_datetime(self.end_week.enddate()))]
                                

        #filter location
        observations = observations[observations['location'] == target_location]

        #aggregate to weekly
        observations = observations.groupby(['location', pd.Grouper(key='date', freq='W-SAT')]).sum().reset_index()

        if self.max_date:
            observations = observations[observations['date'] <= max_date].copy()

        #transform to Observation object
        observations = Observations(observations)

        return observations
    

        
    def get_scenarioensemble(self):
        # calculate scenario ensemble for all locations with quantiles reported in the input df
        predictions = self.df
        
        loclist = list(predictions.location.unique())
        
        
        median_ensembles = pd.DataFrame()
        for l in loclist: # loop through all locations
            
            target_location = l
            
            predictions['target_end_date'] = pd.to_datetime(predictions['target_end_date'])
            
            pred = predictions[predictions['location'] == target_location].copy()
                                               
            target_prediction_list = [f"{i} wk ahead {'inc' if self.inc else 'cum'} {self.target}" for \
                                      i in range(1,len(list(pred.target_end_date.unique()))+1)]
                ## might need to change if data in different format, this is for ensemble^2 rounds 5-16 
        
            #filter target
            pred = pred[(pred['target'].isin(target_prediction_list))]
            
            #filter max date
            if self.max_date:
                pred = pred[pred['target_end_date'] <= self.max_date]

            if len(pred) == 0:
                raise RuntimeError(f"There are no predictions for model {model} at location {target_location}")

            # calculate scenario ensemble
            scenarios = list(pred['scenario_id'].drop_duplicates())
            predictions_list = [Predictions(pred[pred['scenario_id'] == scenario], 
                                            t_col='target_end_date') for scenario in scenarios]
            med_ensemble_predictions = median_ensemble(predictions_list)
            
            med_ensemble_predictions['location'] = target_location
            med_ensemble_predictions['target'] = self.target
                                               
            median_ensembles = pd.concat([median_ensembles, med_ensemble_predictions])
                                               
            predictions_list += [med_ensemble_predictions]
            labels = ["Scenario " + scenario[0] for scenario in sorted(scenarios)] + ["Scenario ensemble"]

        self.scenario_ensemble = median_ensembles
        
        return median_ensembles
          

class Scoring(ScenarioEnsemble):
    # calculate score values for probabilistic epidemic forecasts 
    # find WIS, MAPE, and coverage over whole projection window as well as timestamped for every week.
    # uses scorepi package to calculate the scores 
    # score dataframe must have 'Model' column to differentiate and calculate scores for different models
    
    def __init__(self, df, obsdf, scoredf, target, incidence = True, max_date = False, start_week = False, 
                 end_week = False):
        super().__init__(df, obsdf, target, incidence, max_date, start_week, end_week)
        self.scoredf = scoredf #dataframe we want to use to calculate performance analysis/scoring
        
    def get_all_average_scores(self):
        predictions = self.scoredf.copy()
        #predictions = Predictions(predictions, t_col = 'target_end_date')
        
        allscore = {}
        for model in predictions.Model.unique():
            allscore[model] = {}
            predmod = predictions[predictions.Model==model]
            for target_location in predmod.location.unique():
                observations = self.get_observations(target_location)

                pred = predmod[(predmod.location == target_location) ]
                pred = Predictions(pred, t_col = 'target_end_date')

                d,_ = score_utils.all_scores_from_df(observations, pred, mismatched_allowed=False)

                allscore[model][target_location] = d
            
        
        return allscore
    
    def organize_average_scores(self, want_scores):
        # want_scores is list of scores you want to save in the dataframe
        # wis is 'wis_mean', and all coverages are '10_cov', '20_cov', ... '95_cov' etc.
        
        average_scores = pd.DataFrame()
        
        allscore = self.get_all_average_scores()
        
        for model in allscore.keys():
            scoresmod = allscore[model]
            for loc in scoresmod.keys():
                scoresloc = scoresmod[loc]

                scoredict = {'Model': model ,'location': loc}
                for score in want_scores:
                    scoredict[score] = scoresloc[score]


                average_scores = pd.concat([average_scores, pd.DataFrame(scoredict, index=[0])])

        
        average_scores = average_scores.reset_index() 
        average_scores = average_scores.drop(columns=['index'])
        
        return average_scores
    
    def get_all_timestamped_scores(self):
        predictions = self.scoredf.copy()
        
        allscore = {}
        
        for model in predictions.Model.unique():
            allscore[model] = {}
            predmod = predictions[predictions.Model==model]
            for target_location in predmod.location.unique():
                observations = self.get_observations(target_location)

                pred = predmod[(predmod.location == target_location) ]
                pred = Predictions(pred, t_col = 'target_end_date')

                allscore[model][target_location] = {}
                for t in pred.target_end_date.unique():
                    prednew = pred[pred.target_end_date == t]
                    obsnew = observations[observations.date == t]

                    obsnew = Observations(obsnew)
                    prednew = Predictions(prednew, t_col = 'target_end_date')

                    d = score_utils.all_timestamped_scores_from_df(obsnew, prednew)

                    allscore[model][target_location][t] = d

        #self.allavgscores = allscore
        
        return allscore
    
    def organize_timestamped_scores(self, want_scores):
        # want_scores is list of scores you want to save in the dataframe
        # wis is 'wis'
        
        time_scores = pd.DataFrame()
        
        allscore = self.get_all_timestamped_scores()
        
        for model in allscore.keys():
            scoremod = allscore[model]
        
            for loc in scoremod.keys():
                scoresloc = scoremod[loc]

                for t in scoresloc.keys():
                    tdf = scoresloc[t]

                    scoredict = {'Model':model ,'location':loc, 'target_end_date':t}
                    for score in want_scores:
                        scoredict[score] = tdf[score]


                    time_scores = pd.concat([time_scores, pd.DataFrame(scoredict, index=[0])])

        
        time_scores = time_scores.reset_index() 
        time_scores = time_scores.drop(columns=['index'])
        
        return time_scores
    
    
    def get_rescaled_wis(self):
        # calculate WIS rescaled by standard deviation 
        # need to have WIS scores for multiple models
        
        time_scores = self.organize_timestamped_scores(['wis'])
        
        wisstdev = pd.DataFrame()
        for loc in time_scores.location.unique():
            for date in time_scores.target_end_date.unique():
                df = time_scores[(time_scores.location == loc) & (time_scores.target_end_date == date)]
                
                stdev = df['wis'].std()
                
                df['wis_scaled'] = df['wis'] / stdev
                
                wisstdev = pd.concat([wisstdev, df])
                
        wisstdev = wisstdev.reset_index()
        wisstdev = wisstdev.drop(columns=['index'])
        
        return wisstdev
    
    
    def get_wis_ratios(self, numerator_model, denominator_model, timestamped=False):
        # need all models of interest in score df so we can calculate wis of each
        # input is numerator model and denominator model to calculate wis ratio
        # input whether you want ratio taken at each time point or over average of whole time window
        
        if timestamped == True:
            scores = self.organize_timestamped_scores(['wis'])
        else:
            scores = self.organize_average_scores(['wis_mean'])
            scores = scores.rename(columns={'wis_mean':'wis'})
            
        num = scores[scores.Model == numerator_model]
        num = num.rename(columns={"wis": "wis_num"})
        
        denom = scores[scores.Model == denominator_model]
        denom = denom.rename(columns={"wis": "wis_denom"})
        
        if timestamped == True:
            scoresmerge = pd.merge(num, denom, how='inner', on = ['location', 'target_end_date'])
        else:
            scoresmerge = pd.merge(num, denom, how='inner', on = ['location'])
            
        scoresmerge['wis_ratio'] = scoresmerge['wis_num'] / scoresmerge['wis_denom']
        
        return scoresmerge
        
    
    
#other things to code up:
    #  make plots?

In [19]:
df = pd.read_parquet(f'./dat/Ensemble_rd12.pq')
df

,scenario_id,scenario_name,target,target_end_date,location,value,type,quantile,model_projection_date
0,A-2022-01-09,optSev_highIE,1 wk ahead cum case,2022-01-15,01,1.041686e+06,quantile,0.01,2022-01-09
1,A-2022-01-09,optSev_highIE,1 wk ahead cum case,2022-01-15,02,1.764332e+05,quantile,0.01,2022-01-09
2,A-2022-01-09,optSev_highIE,1 wk ahead cum case,2022-01-15,04,1.550991e+06,quantile,0.01,2022-01-09
3,A-2022-01-09,optSev_highIE,1 wk ahead cum case,2022-01-15,05,6.631201e+05,quantile,0.01,2022-01-09
4,A-2022-01-09,optSev_highIE,1 wk ahead cum case,2022-01-15,06,6.453597e+06,quantile,0.01,2022-01-09
...,...,...,...,...,...,...,...,...,...
377563,D-2022-01-09,pessSev_lowIE,9 wk ahead inc hosp,2022-03-12,66,0.000000e+00,quantile,0.99,2022-01-09
377564,D-2022-01-09,pessSev_lowIE,9 wk ahead inc hosp,2022-03-12,69,0.000000e+00,quantile,0.99,2022-01-09
377565,D-2022-01-09,pessSev_lowIE,9 wk ahead inc hosp,2022-03-12,72,1.183000e+02,quantile,0.99,2022-01-09
377566,D-2022-01-09,pessSev_lowIE,9 wk ahead inc hosp,2022-03-12,78,5.950000e+01,quantile,0.99,2022-01-09


In [127]:
incidence = True
target_obs = 'death'
obsdf = pd.read_parquet(f"./dat/truth_{'inc' if incidence else 'cum'}_{target_obs}.pq")

dfall = pd.DataFrame()
for model in ['Ensemble', 'Ensemble_LOP', "MOBS_NEU-GLEAM_COVID", "JHU_IDD-CovidSP"]:
    df = pd.read_parquet(f'./dat/{model}_rd12.pq')
    test = ScenarioEnsemble(df=df, obsdf=obsdf, target='death', incidence = True, max_date = False, 
                            start_week = Week(2022,2), end_week = Week(2022, 13))
    medens = test.get_scenarioensemble()
    medens['Model'] = model

    dfall = pd.concat([dfall, medens])



In [128]:
dfall

,target_end_date,type,quantile,value,location,target,Model
0,2022-01-15,quantile,0.010,62.110704,01,death,Ensemble
1,2022-01-15,quantile,0.990,276.116947,01,death,Ensemble
2,2022-01-15,quantile,0.975,237.251311,01,death,Ensemble
3,2022-01-15,quantile,0.950,222.625229,01,death,Ensemble
4,2022-01-15,quantile,0.900,212.500905,01,death,Ensemble
...,...,...,...,...,...,...,...
283,2022-04-02,quantile,0.010,0.000046,56,death,JHU_IDD-CovidSP
284,2022-04-02,point,NaN,0.367829,56,death,JHU_IDD-CovidSP
285,2022-04-02,quantile,0.975,10.601108,56,death,JHU_IDD-CovidSP
286,2022-04-02,quantile,0.400,0.180596,56,death,JHU_IDD-CovidSP


In [157]:
test2 = Scoring(df=df, obsdf=obsdf,scoredf = dfall, 
               target='death', incidence = True, max_date = False, 
                        start_week = Week(2022,2), end_week = Week(2022, 13))
d = test2.get_wis_ratios(numerator_model='Ensemble', denominator_model = 'MOBS_NEU-GLEAM_COVID', timestamped=False)

In [158]:
d

,Model_x,location,wis_num,Model_y,wis_denom,wis_ratio
0,Ensemble,01,128.230156,MOBS_NEU-GLEAM_COVID,97.125650,1.320250
1,Ensemble,02,9.503650,MOBS_NEU-GLEAM_COVID,8.495319,1.118692
2,Ensemble,04,238.160749,MOBS_NEU-GLEAM_COVID,230.067397,1.035178
3,Ensemble,05,93.472753,MOBS_NEU-GLEAM_COVID,113.194370,0.825772
4,Ensemble,06,600.166135,MOBS_NEU-GLEAM_COVID,596.323739,1.006443
5,Ensemble,08,20.097183,MOBS_NEU-GLEAM_COVID,22.348595,0.899259
6,Ensemble,09,32.745176,MOBS_NEU-GLEAM_COVID,31.858146,1.027843
7,Ensemble,10,17.716735,MOBS_NEU-GLEAM_COVID,17.718005,0.999928
8,Ensemble,11,2.843929,MOBS_NEU-GLEAM_COVID,3.943376,0.721191
9,Ensemble,12,647.047395,MOBS_NEU-GLEAM_COVID,254.019644,2.547234
